In [1]:
from datetime import datetime
from meteostat import Point, Hourly
import pandas as pd
import os


start = datetime(2025, 1, 1)
end = datetime(2025, 1, 31, 23, 59)


airport_coords = {
    'ATL': (33.6407, -84.4277),  # Hartsfield-Jackson Atlanta
    'DFW': (32.8998, -97.0403),  # Dallas/Fort Worth
    'DEN': (39.8561, -104.6737), # Denver
    'ORD': (41.9742, -87.9073),  # Chicago O'Hare
    'LAX': (33.9416, -118.4085), # Los Angeles
    'JFK': (40.6413, -73.7781),  # New York JFK
    'LAS': (36.0840, -115.1537), # Harry Reid (Las Vegas)
    'MCO': (28.4312, -81.3081),  # Orlando
    'MIA': (25.7959, -80.2870),  # Miami
    'CLT': (35.2144, -80.9473),  # Charlotte Douglas
    'SEA': (47.4502, -122.3088), # Seattle-Tacoma
    'SFO': (37.6213, -122.3790)  # San Francisco
}

dfs = []
print(f">>> Starting data download from {start} to {end}...")

for code, (lat, lon) in airport_coords.items():
    try:
        print(f"Downloading weather for {code}...", end=" ")
        
        location = Point(lat, lon)
        
        data = Hourly(location, start, end)
        data = data.fetch()
        
        if data.empty:
            print("[WARNING] No data found!")
            continue

        data = data.reset_index()
        data['Airport'] = code  
        

        cols_to_keep = ['time', 'Airport', 'temp', 'rhum', 'prcp', 'wspd', 'coco']
        
        available_cols = [c for c in cols_to_keep if c in data.columns]
        data = data[available_cols]
        
        dfs.append(data)
        print("OK.")
        
    except Exception as e:
        print(f"\n[ERROR] Error downloading {code}: {e}")

if dfs:
    weather_final = pd.concat(dfs)
    
    output_dir = "../data/raw"
    os.makedirs(output_dir, exist_ok=True)
    
    output_path = f"{output_dir}/weather_2025_01.csv"
    weather_final.to_csv(output_path, index=False)
    
    print("-" * 30)
    print(f">>> Done! File saved at: {output_path}")
    print(f">>> Total rows: {len(weather_final)}")
    print(weather_final.head())
else:
    print(">>> No data could be downloaded.")


>>> Starting data download from 2025-01-01 00:00:00 to 2025-01-31 23:59:00...

OK.

OK.
------------------------------
>>> Done! File saved at: ../data/raw/weather_2025_01.csv
>>> Total rows: 8928
                 time Airport  temp  rhum  prcp  wspd  coco
0 2025-01-01 00:00:00     ATL  12.0  50.0   0.0  30.0   1.0
1 2025-01-01 01:00:00     ATL  11.7  50.0   0.0  29.5   1.0
2 2025-01-01 02:00:00     ATL  10.6  52.0   0.0  20.5   1.0
3 2025-01-01 03:00:00     ATL  10.0  54.0   0.0  24.1   3.0
4 2025-01-01 04:00:00     ATL   9.4  52.0   0.0  22.3   1.0
